# Adding information to BdHNe table

One of these days I published the online version of ICRANet Binary-driven Hyper-Novae table of Gamma-Ray Bursts (GRB) at the (BSDC/VO)[http://vo.bsdc.icranet.org] website; the table: http://vo.bsdc.icranet.org/bdhne/q/web/form.
The (current) first version of the table contains exactly what is being published in the article (Ruffini et al, in preparation), which is:

* GRB (id): the standard name for GRBs (*YYMMDD*[A-Z]);
* Designation: as proposed by the authors of the article;
* z: redshift;
* E_iso: the energy computedby the authors after their model;
* Instrument: observatory from where data was used;
* GCN: the GCN corresponding to the event.

What I'm going to do now is to add position (Right Ascension and Declination) and trigger time of those events.
To do so, I should retrieve such information from online services/tables and join them with the current table.

Swift GRBs data will be used to get such information (position and trigger time). 
(*"--Why Swift? --Because I had to choose one; next time will be Fermi."*)
After a search for which website could give me better the information, the [Mission's American website][NASAGRB].
Through their service I can easily automate the query, retrieve and parsing process, plus they provide all the relevant information I am looking for (almost) all GRBs.
For the records, below is the list of websites I looked into; The list was kindly provided by my colleaghe Milos Kovacevic:

* http://www.swift.ac.uk/xrt_positions/
* https://heasarc.gsfc.nasa.gov/W3Browse/fermi/fermigbrst.html
* https://heasarc.gsfc.nasa.gov/W3Browse/all/batsegrb.html
* http://www.astro.caltech.edu/grbox/grbox.php
* (**Chosen**) https://swift.gsfc.nasa.gov/archive/grb_table/

Data and software we already have (properly working/formatted):

* The current BdHNe table is here copied as filename `ibdhne.csv`;
* The Perl script to retrieve the table from [Swift GRB service][NASAGRB], called `SwiftGRBsNASA.pl`;
 * We will execute the script to generate an up-to-date table of GRBs named `SwiftGRBsNASA_table.csv`.

[NASAGRB]: https://swift.gsfc.nasa.gov/archive/grb_table/